In [1]:
!pip install torch datasets transformers

You should consider upgrading via the '/lnet/aic/personal/kydliceh/non_runable/NLP_venv/bin/python3 -m pip install --upgrade pip' command.


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")

In [4]:
from transformers import AutoModelForSequenceClassification
import torch.nn as nn
def grad_params(params, freeze=True):
    for param in params:
        param.requires_grad = freeze



def get_model(out_dim=5, freeze_layers=2):
    model = AutoModelForSequenceClassification.from_pretrained("ufal/robeczech-base", num_labels=out_dim)
    grad_params(model.parameters(), freeze=True)
    grad_params(model.classifier.parameters(), freeze=False)
    for layer in model.roberta.encoder.layer[-freeze_layers:]:
        grad_params(layer.parameters(), freeze=False)
    return model



In [5]:
from datasets import load_dataset

In [6]:
eval_dataset = load_dataset("../../../final_dataset/dataset.py", split="validation")

Found cached dataset dataset (/home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/3a21ebffd6f7380b097ffe2f8be2beda23497fd4b6ff06de19b9c473635bb199)


In [7]:
train_dataset = load_dataset("../../../final_dataset/dataset.py", split="train")
train_dataset = train_dataset.remove_columns(["category"])

Found cached dataset dataset (/home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/3a21ebffd6f7380b097ffe2f8be2beda23497fd4b6ff06de19b9c473635bb199)


In [8]:
tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")
def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, max_length=512)

train_dataset_new = train_dataset.map(preprocess_function, batched=True)

The OrderedVocab you are attempting to save contains a hole for index 51959, your vocabulary could be corrupted !


  0%|          | 0/1784 [00:00<?, ?ba/s]

In [9]:
eval_dataset = eval_dataset.remove_columns(["category"])
eval_dataset_new = eval_dataset.map(preprocess_function, batched=True)

The OrderedVocab you are attempting to save contains a hole for index 51959, your vocabulary could be corrupted !


  0%|          | 0/223 [00:00<?, ?ba/s]

In [10]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(max_length=512, tokenizer=tokenizer)

In [11]:
def prepare_dataset(x, column):
    x = x.filter(lambda ex: ex[column] is not None)
    x = x.rename_column(column, "labels")
    return x

In [12]:
server_dataset = prepare_dataset(train_dataset_new, "server")

  0%|          | 0/1784 [00:00<?, ?ba/s]

In [13]:
server_eval_dataset = prepare_dataset(eval_dataset_new, "server")

  0%|          | 0/223 [00:00<?, ?ba/s]

In [15]:
import torch
import gc
from transformers import TrainingArguments, Trainer
def train_classification(name: str, model, dataset, eval_dataset, epochs=2, batch_size=10):
    torch.cuda.empty_cache()
    gc.collect()
    training_args = TrainingArguments(output_dir=f"./{name}", overwrite_output_dir=True, num_train_epochs=2, per_gpu_train_batch_size=batch_size, per_gpu_eval_batch_size=batch_size,
    logging_dir=f"./logs/{name}", logging_steps=1000, save_steps=10000, save_total_limit=5, evaluation_strategy="steps", eval_steps=10000, learning_rate=2e-5, warmup_steps=100, weight_decay=0.05, adam_beta1=0.9, adam_beta2=0.999, warmup_ratio=0.1)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        compute_metrics=lambda x: {"accuracy": (x.predictions.argmax(-1) == x.label_ids).mean()},
    )
    trainer.train()


In [ ]:
model = get_model(6)
train_classification("server", model, server_dataset, server_eval_dataset)

In [17]:
day_of_week_dataset = prepare_dataset(train_dataset_new, "day_of_week")
day_of_week_eval_dataset = prepare_dataset(eval_dataset_new, "day_of_week")

Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/3a21ebffd6f7380b097ffe2f8be2beda23497fd4b6ff06de19b9c473635bb199/cache-3584e9292ab6e185.arrow
Loading cached processed dataset at /home/kydliceh/.cache/huggingface/datasets/dataset/default/0.0.0/3a21ebffd6f7380b097ffe2f8be2beda23497fd4b6ff06de19b9c473635bb199/cache-d237c5eda9ff5747.arrow


In [16]:
model_day = get_model(7)
train_classification("day_of_week", model_day, day_of_week_dataset, day_of_week_eval_dataset)

Some weights of the model checkpoint at ufal/robeczech-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ufal/robeczech-base and are newly initialized: 

NameError: name 'model' is not defined